In [ ]:
## The Saltenis estimator

One starts off by importing the relevant libraries and functions. The analytical values of the total indices are also imported so as to benchmark the values estimated.

%matplotlib inline
from Functions import A1, A2, A3, B1, B2, B3, C1, C2
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from pandas import ExcelWriter
plt.style.use('ggplot')

k = 6

a2 = np.array([0,0.5,3,9,99,99])
b3 = np.array([6.42,6.42,6.42,6.42,6.42,6.42])

functions = [A1, A2, A3, B1, B2, B3, C1, C2]

AE_df = pd.read_excel('AE_df.xlsx',index_col=0) #the analytical values

def scrambler(q,s):
    return q+s

def Saltenis(a,b):
    return 0.5*(a-b)**2

The low-discrepancy sample matrix is then imported along with a scrambling rule so as to produce 50 independent repetions one can work with to produce a stable estimate.

In [2]:
p = 14
run = 50

n = 2

df = pd.DataFrame(sobol_seq.i4_sobol_generate(6*k,-1+2**p))

df_index = pd.read_csv('2019.10.08_index.csv', index_col=0)

df_r = pd.concat([df.T.reindex(df_index.iloc[r]).reset_index(drop=True).T for r in range(run)])

The mean absolute errors are finally estimated across functions

In [25]:
AE = {f.__name__:[] for f in functions}
AE_f = {f.__name__:[] for f in functions}
MAE = {f.__name__:[] for f in functions}

RMSE = {f.__name__:pd.Series() for f in functions}
SE = {f.__name__:[] for f in functions}
SE_f = {f.__name__:[] for f in functions}

RE = {f.__name__:[] for f in functions}
RE_f = {f.__name__:[] for f in functions}

for f in functions:
    elementary_effect = []
    ea = pd.DataFrame(f(df_r.iloc[:,:k]))
    ee_df =[]
    for j in range(k):
        rs = df_r.iloc[:,:2*k].copy()
        rs[j] = rs[scrambler(j,k)] # generate the scrambled matrices
        ee = pd.DataFrame(0.5*(f(df_r.iloc[:,:k])-f(rs.iloc[:,:k]))**2,columns=[j]) # compute the elementary effects
        ee['r']= [r for r in range(run) for rdf in range(len(df))]
        ee_df.append(ee[j])
        
        elementary_effect.append(ee.groupby('r')[j].expanding(1).mean()) # row-wise average along individual repetitions

    ea['r']=ee['r']
    elementary_effect_df = pd.concat(elementary_effect,axis=1)
    Var_df = ea.groupby('r')[0].expanding(1).var(ddof=0) # compute the row-wise variance for each repetition
    
    AE[f.__name__] = np.abs(elementary_effect_df.div(Var_df,axis=0) - AE_df.loc[f.__name__]) # assess the absolute error for each factor
    SE[f.__name__] = (elementary_effect_df.div(Var_df,axis=0) - AE_df.loc[f.__name__])**2 # assess the root square error
    RE[f.__name__] = AE[f.__name__].div(AE_df.loc[f.__name__]) # assess the relative abs error
    
    AE[f.__name__].index = AE[f.__name__].index.droplevel(0)
    SE[f.__name__].index = AE[f.__name__].index
    RE[f.__name__].index = AE[f.__name__].index
    
    AE_f[f.__name__] = AE[f.__name__].groupby(AE[f.__name__].index).mean() #average over the 50 repetitions
    SE_f[f.__name__] = np.sqrt(SE[f.__name__].groupby(SE[f.__name__].index).mean())
    RE_f[f.__name__] = RE[f.__name__].groupby(RE[f.__name__].index).mean()
    
    MAE[f.__name__] = AE_f[f.__name__].mean(axis=1) # compute the mean absolute error by averaging over the factors
    RMSE[f.__name__] = SE_f[f.__name__].mean(axis=1) # compute the mean root square error by averaging over the factors
    
    MAE[f.__name__].index = (MAE[f.__name__].index+1)*(k+1) # set the index according to the total cost
    RMSE[f.__name__].index = MAE[f.__name__].index
    RE_f[f.__name__].index = MAE[f.__name__].index
    
#export the data for comparisons and plotting

writer = pd.ExcelWriter('Saltenis_asym.xlsx', engine='xlsxwriter')
for mk in MAE.keys():
    MAE[mk].to_excel(writer, sheet_name=mk)
writer.save()

writer = pd.ExcelWriter('Saltenis_asym_RMSE.xlsx', engine='xlsxwriter')
for mk in RMSE.keys():
    RMSE[mk].to_excel(writer, sheet_name=mk)
writer.save()

writer = pd.ExcelWriter('Saltenis_asym_RE.xlsx', engine='xlsxwriter')
for mk in RE_f.keys():
    RE_f[mk].loc[[(k+1)*(-1+2**s) for s in range(1,p)]].round(3).to_excel(writer, sheet_name=mk)
writer.save()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """
